<a href="https://colab.research.google.com/github/wrightky/ANUGA_OnionCreek_Tutorial/blob/master/notebooks/notebook1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ANUGA Lab

## CE377K - River Mechanics
### Kyle Wright
#### March 29-30, 2022

The goal of this Notebook is to give users an introduction to some of the basic functionality of the **ANUGA** hydrodynamic model. This model solves the 2D shallow-water equations on an **unstructured triangular grid**, and offers some key advantages over other models for modeling the hydrodynamics of Earth systems.

In this tutorial, we will build a simple, small, coarse model of the **Onion Creek Watershed**. Different cell blocks will demonstate how one can build the model mesh, initialize the domain, setup the parameters of the model, and run the simulation. 

The point of this script is not to build the best model we can of the domain, but rather to **demonstrate the workflow** of constructing a model that can be modified or replicated for other (better) models. 

These notebooks are written in Python 3, and have been designed to run in the Google `colaboratory` environment, which provides a Jupyter notebook environment running on a virtual machine on the cloud. Nothing needs to be installed locally, as it is entirely run on Google Drive. All you need is a Google account. All of these scripts are stored on GitHub, which is a platform for sharing open-source codes.

To start interacting with the notebook follow the `Open in Colab` link above. Once you're there, click `File` > `Save a Copy in Drive`

## Setup Environment

The first thing we have to do is set up our environment. In Python, this means downloading all the necessary functions we want to use, including ANUGA. 

Run the following cell to install the dependencies and some extra code for visualising on Colaboratory. To run each code block, you can simply type `shift` + `enter`

Wait until you see the comment *(5) Ready to go* before proceeding to subsequent commands. The install should take less than a minute.

NOTE: Tutorial installation is based on the [2018 CSDMS Anuga Clinic](https://github.com/stoiver/anuga-clinic-2018)

In [ ]:
# 
try:
  import os
  os.chdir('/content')
  !git clone https://github.com/wrightky/ANUGA_OnionCreek_Tutorial.git

  # Now install environment using tool
  !/bin/bash /content/ANUGA_OnionCreek_Tutorial/anuga_tools/install_anuga_colab.sh
 
except:
  pass

# Make anuga-clinic code available

if not 'workbookDir' in globals():
    workbookDir = os.getcwd()

import sys
sys.path.append(os.path.join(workbookDir,"ANUGA_OnionCreek_Tutorial"))